In [ ]:
!pip install pgmpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 10.2 MB/s eta 0:00:00


In [ ]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.sampling import BayesianModelSampling
from pgmpy.inference import VariableElimination

# Create a Bayesian Network
model = BayesianNetwork([('S', 'C'), ('R', 'C'), ('F', 'R'), ('M', 'R'), ('S', 'Y')])

# Define Conditional Probability Distributions (CPDs)
cpd_s = TabularCPD(variable='S', variable_card=2, values=[[0.7], [0.3]])
cpd_f = TabularCPD(variable='F', variable_card=2, values=[[0.2], [0.8]])
cpd_m = TabularCPD(variable='M', variable_card=2, values=[[0.1], [0.9]])

cpd_r = TabularCPD(variable='R', variable_card=2,
                   values=[[0.9, 0.8, 0.8, 0.1],
                           [0.1, 0.2, 0.2, 0.9]],
                   evidence=['F', 'M'], evidence_card=[2, 2])

cpd_c = TabularCPD(variable='C', variable_card=2,
                   values=[[0.9, 0.3, 0.6, 0.1],
                           [0.1, 0.7, 0.4, 0.9]],
                   evidence=['S', 'R'], evidence_card=[2, 2])
cpd_y = TabularCPD(variable='Y', variable_card=2,
                   values=[[0.89, 0.8],
                           [0.11, 0.2]],
                   evidence=['S'], evidence_card=[2])


# Add CPDs to the model
model.add_cpds(cpd_s, cpd_f, cpd_m, cpd_r, cpd_c, cpd_y)

# Check the model for consistency
assert model.check_model()

inference = BayesianModelSampling(model)
num_samples = 10000
samples = inference.likelihood_weighted_sample(size=num_samples)
count_y = sum(samples["Y"])
print(f'probability of yellow fingers: {count_y/num_samples}')

infer = VariableElimination(model)
q=infer.query(variables=["S", "F", "M", "Y"], evidence={"R":1, "C":1})
print(q)
print(f'Hence, the most probable explanation for R = 1, C = 1 is S = 0, F = 0, M = 0, Y = 0')

  0%|          | 0/6 [00:00<?, ?it/s]

probability of yellow fingers: 0.1346
+------+------+------+------+----------------+
| S    | F    | M    | Y    |   phi(S,F,M,Y) |
+======+======+======+======+================+
| S(0) | F(0) | M(0) | Y(0) |         0.0016 |
+------+------+------+------+----------------+
| S(0) | F(0) | M(0) | Y(1) |         0.0002 |
+------+------+------+------+----------------+
| S(0) | F(0) | M(1) | Y(0) |         0.0294 |
+------+------+------+------+----------------+
| S(0) | F(0) | M(1) | Y(1) |         0.0036 |
+------+------+------+------+----------------+
| S(0) | F(1) | M(0) | Y(0) |         0.0131 |
+------+------+------+------+----------------+
| S(0) | F(1) | M(0) | Y(1) |         0.0016 |
+------+------+------+------+----------------+
| S(0) | F(1) | M(1) | Y(0) |         0.5297 |
+------+------+------+------+----------------+
| S(0) | F(1) | M(1) | Y(1) |         0.0655 |
+------+------+------+------+----------------+
| S(1) | F(0) | M(0) | Y(0) |         0.0008 |
+------+------+------+